In [ ]:
import csv
import sys
import numpy as np
import os
# Set field size limit to the maximum possible
csv.field_size_limit(sys.maxsize)
import ast
import random
import matplotlib.pyplot as plt
import time
import pickle
import gc
from read_data import *
from gpcam import GPOptimizer
from datetime import datetime
import shutil
from functools import partial
import dask
from dask.distributed import Client
import os
import time
from gpAMRutils import *

## Tuning options to consider
- Filtering data by block or as a function of the global data?
- What fraction of candidates should be used, > mean SD? top 10%, top n candidates?
- How will data noise be estimated and set?
- RAM: submitting ask several times means that $\kappa \in \mathbb{R}^{12000 \times 12000}$ is possibly stored many times at once. At the same time the interpolator in the kernel stores temporarily 2(3) matrices of shape (len(x_data)^2)

## SETTINGS

In [ ]:
#settings:
number_of_workers = 256
number_of_threads = 32

tol_ratio = 1e-2 #remove data < tol_ratio * max(data)
refinement_res = 2 #refinement level
plotting = True #plot posterior mean and var
plot_resx = 100
plot_resy =  50

#filename = "plot.nx256.2d.AMRLevel.hdf5"
filename = "plot.nx2048.2d.AMRLevel.hdf5"
index = "vorticity"
percentile = 99 ##of all candidates considered, return the top percentile

chombo_path = "./ChomboOut/"
gpcam_path = "./gpCAMOut/"


init_hyperparameters = np.array([1., 20., 0.5])
hyperparameter_bounds = np.array([[0.1, 10.],
                                  [0.1, 10.],
                                  [0.1, 0.5],])

if os.path.exists(chombo_path+"ready.txt") and os.path.exists(chombo_path+filename):
    print("A data file already exists in the repo. Should I delete it? y/n")
    dec = input()
    if dec  == "y":
            print("File removed ...")
            os.remove(chombo_path+filename)
            os.remove(chombo_path+"ready.txt")
    elif dec == "n": print("file not deleted")
    else: print("This was not a viable option, run the cell again!")

## Dask Client

In [ ]:
scheduler_file = os.path.join(os.environ["SCRATCH"], "scheduler_filegpAMR.json")
dask.config.config["distributed"]["dashboard"]["link"] = "{JUPYTERHUB_SERVICE_PREFIX}proxy/{host}:{port}/status" 

client = init_client(scheduler_file, number_of_workers)

In [ ]:
client

## GP set-up

In [ ]:
from scipy.interpolate import griddata
from gpcam.kernels import *
from scipy.interpolate import RBFInterpolator
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import spsolve
from scipy.spatial.distance import cdist

# Wendland C^2 compactly supported kernel (support radius = 1)
def wendland_c2(r):
    out = np.zeros_like(r)
    mask = r < 1.0
    rm = 1 - r[mask]
    out[mask] = rm**4 * (4*r[mask] + 1)
    return out


# Derivative of Wendland C^2 w.r.t. r
def wendland_c2_prime(r):
    out = np.zeros_like(r)
    mask = r < 1.0
    rm = 1 - r[mask]
    # φ'(r) = d/dr [ (1-r)^4 * (4r +1) ]
    #       = -4*(1-r)^3*(4r+1) + (1-r)^4 * 4
    out[mask] = -4 * rm**3 * (4*r[mask] + 1) + 4 * rm**4
    return out

# Simple RBF interpolator with Wendland kernel
class WendlandRBF:
    def __init__(self, x, y, epsilon=0.5):
        self.x = np.asarray(x)
        self.y = np.asarray(y)
        self.epsilon = epsilon
        r = cdist(self.x, self.x) * epsilon
        K = wendland_c2(r)
        K = csr_matrix(K)
        nnz = K.nnz

        # Total number of entries
        total = np.prod(K.shape)

        # Sparsity = fraction of zeros
        sparsity = nnz / total
        self.w = spsolve(K, self.y)

    def __call__(self, xnew):
        r = cdist(np.atleast_2d(xnew), self.x) * self.epsilon
        K = wendland_c2(r)
        K = csr_matrix(K)
        return np.asarray(K @ self.w)
        
    def gradient(self, xnew):
        """
        Compute the gradient of the RBF interpolant at one or multiple query points.
        xnew: array of shape (M, d)
        Returns: array of shape (M, d)
        """
        xnew = np.atleast_2d(xnew)  # (M, d)
        M, d = xnew.shape
        N = self.x.shape[0]
    
        grads = np.zeros((M, d))
    
        for j in range(M):
            diffs = xnew[j] - self.x        # (N, d)
            r = np.linalg.norm(diffs, axis=1) * self.epsilon  # (N,)
            phi_prime = wendland_c2_prime(r)
    
            grad = np.zeros(d)
            for i in range(N):
                if 0 < r[i] < 1.0:
                    grad += self.w[i] * phi_prime[i] * self.epsilon * (diffs[i] / (r[i] + 1e-12))
    
            grads[j] = grad
    
        return np.linalg.norm(grads, axis = 1)


def kernelPDE(x1, x2, hps, x_data = None, y_data = None):
    #st = time.time()
    rbf = WendlandRBF(x_data[::2], y_data[::2], epsilon = hps[2]/2.)
    #print(time.time() - st, flush = True)
    func1 = abs(rbf(x1))
    func2 = abs(rbf(x2))

    #func1 = abs(rbf.gradient(x1))
    #func2 = abs(rbf.gradient(x2))
    del rbf
    d = get_distance_matrix(x1,x2)
    k = hps[0] * np.outer(func1,func2) * matern_kernel_diff1(d,hps[1])
    return k 

def meanf(x,hps):
    return np.zeros(len(x))

def acq_func(x, gp):
    x = np.asarray(x)
    return np.sqrt(gp.posterior_covariance(x, variance_only=True)["v(x)"])

In [ ]:
datasets, domain, global_x, global_y = read_file(chombo_path, filename, index, tol_ratio)
init_x_data = np.array([[domain[0,0], domain[1,0]],[domain[0,0], domain[1,1]],[domain[0,1], domain[1,0]],[domain[0,1], domain[1,1]]])
init_y_data = np.array([0., 0., 0., 0.])
tol = tol_ratio * np.max(abs(global_y))

GPs = {}
candidate_pools = {}
block_domains = {}
kernels = {}
print("Initializing GPs")
for ID in datasets:
    assert len(datasets[ID][0]) == len(datasets[ID][1])
    #print(ID, end = ' ')
    if len(datasets[ID][0]) >= 4:
        #print("active", end = "....;   ")
        #comm data to kernel func.
        kernels[ID] = partial(kernelPDE, x_data = global_x, y_data = global_y)
        #init GPs
        GPs[ID] = GPOptimizer(datasets[ID][0], datasets[ID][1], noise_variances=np.ones(datasets[ID][1].shape) * 0.1,
                          kernel_function=kernels[ID],
                          prior_mean_function=meanf,
                          init_hyperparameters = init_hyperparameters,
                          args={"active": True}, calc_inv=True)

    else:
        #comm data to kernel func.
        #print("inactive", end = "....;   ")
        kernels[ID] = partial(kernelPDE, x_data = global_x, y_data = global_y)
        #init GPs
        GPs[ID] = GPOptimizer(init_x_data, init_y_data, noise_variances=np.ones(init_y_data.shape) * 0.1,
                          kernel_function=kernels[ID],
                          prior_mean_function=meanf,
                          init_hyperparameters = init_hyperparameters,
                          args={"active": False}, calc_inv=True)

    #define refinement res
    xmin = datasets[ID][2][0,0]
    xmax = datasets[ID][2][0,1]
    ymin = datasets[ID][2][1,0]
    ymax = datasets[ID][2][1,1]
    grid = np.array(np.meshgrid(np.linspace(xmin, xmax, refinement_res * int(xmax-xmin) + 1)[:-1], np.linspace(ymin, ymax, refinement_res * int(ymax-ymin) + 1)[:-1])).T.reshape(-1, 2)
    mask = np.all(np.isclose(grid, np.round(grid)), axis=1) #filter out all existing data grid points, THIS ONLY WORKS IF DATA POINTS ARE INTEGERS
    grid = grid[~mask] #filter out all existing data grid points
    candidate_pools[ID] = [array for array in grid]
    if not valid(candidate_pools[ID]): raise Exception("Invalid candidate pool in ID ", ID)
    block_domains[ID] = datasets[ID][2]
print("done!")


all_candidates = np.vstack([np.asarray(candidate_pools[ID]) for ID in datasets])
if not valid(all_candidates): raise Exception("Duplicates in global candidate pool")


GPs = client.scatter(GPs, broadcast=False, direct=True)
print("Initial training...")
train_futures = []
for ID in datasets: 
    train_futures.append(train(client, hyperparameter_bounds, GPs[ID],  max_iter = 1000, method = "mcmc"))
client.gather(train_futures)
print("done!")



###################################################
###################################################
iteration_counter = 0
training_at = [1,2,3,4,5, 10]
print("#######################")
print("start gpAMR iteration: ")
print("#######################")
suggestion_history = []
while True:
    iteration_counter += 1
    print("")
    print("")
    print("++++++++++++++++++++++++++++++++++")
    print("start gpAMR iteration: ", iteration_counter)
    print("++++++++++++++++++++++++++++++++++")
    
    res = []
    #ASKING FOR SUGGESTIONS
    print("Asking for new suggestions")
    for ID in datasets:
        if not GPs[ID].result().args["active"]: continue
        print("        Asking GP ",ID, " with ",len(GPs[ID].result().x_data)," data points, for suggestions")
        candidate_pool = candidate_pools[ID]
        #candidates = list(chunks(candidate_pool, number_of_threads))
        print("        ask for new suggestions.... Candidates  considered: ", len(candidate_pool))
        #for chunk in candidates: res.append(ask(client, chunk, GPs[ID], len(chunk), acq_func)) ###FAST BUT RAM INEFFICIENT
        #print(np.asarray(candidate_pool))
        res.append(ask(client, candidate_pool, GPs[ID], len(candidate_pool), acq_func)) ###SLOWER BUT RAM EFFICIENT
    new = client.gather(res)

    print("    All GP agents reported suggestions...concatenating")
    SD = np.concatenate([x["f_a(x)"] for x in new])
    new = np.vstack([x["x"] for x in new])
    sorted_indices = np.argsort(SD)[::-1]
    SD = SD[sorted_indices]
    new = new[sorted_indices]
    uncertainty_tol = 0.01 #np.percentile(SD, percentile) ##NEEDS IMPROVEMENTS
    plt.plot(SD)
    print("uncertainty tol: ", uncertainty_tol)
    plt.show()
    non_zero_ind = np.where(SD > uncertainty_tol)
    if not valid(new): raise Exception("Non-Unique suggestions")
    suggestions = new[non_zero_ind]
    print("len(suggestions): ", len(suggestions))
    print("    Suggestions calculated, len:", len(suggestions))
    print("    Write suggestions for Chombo iteration... ", iteration_counter)
    #################################
    #suggestions = np.round(suggestions)
    #################################
    if not valid(suggestions): raise Exception("Non-Unique suggestions")
    write_file(gpcam_path, chombo_path, suggestions) ##send to data generator (simulation)
    print("    Suggestions written!")
    

    suggestion_history.append(suggestions)
    

    #PLOTTING
    if plotting:
        print("Generating plots...")
        mean_futures = []
        cov_futures = []
        for ID in datasets:
            if not GPs[ID].result().args["active"]: continue 
            x_plot = GPOptimizer.make_2d_x_pred(block_domains[ID][0],block_domains[ID][1],resx=plot_resx,resy=plot_resy)
            mean_futures.append(posterior_mean(client, x_plot,GPs[ID]))
            cov_futures.append(posterior_covariance(client, x_plot,GPs[ID]))
        means_tmp = client.gather(mean_futures)
        means = np.concatenate([mean["m(x)"] for mean in means_tmp])
        cov_tmp = client.gather(cov_futures)
        stds = np.concatenate([np.sqrt(cov["v(x)"]) for cov in cov_tmp])
        x_pred = np.vstack([mean["x_pred"] for mean in means_tmp])
        
        print("data and suggestions:")
        plt.figure(figsize=(20,5))
        a = plt.scatter(global_x[:,0], global_x[:,1], c=global_y, alpha=.2)
        plt.scatter(suggestions[:,0], suggestions[:,1], s=0.1, c='black', alpha=1.)
        plt.colorbar(a)
        plt.show()
        
        plt.figure(figsize=(20,5))
        plot2d(x_pred[:,0], x_pred[:,1], means, suggestions=suggestions, title= "mean and suggestions",filename=gpcam_path+"mean" + str(iteration_counter).zfill(4))
        
        print("std and suggestions:")
        plot2d(x_pred[:,0], x_pred[:,1], stds, suggestions=suggestions, title= "std and suggestions", filename=gpcam_path+"std" + str(iteration_counter).zfill(4))

        ##write image to disc
        plt.figure(figsize=(20,5))
        a = plt.scatter(x_pred[:,0],x_pred[:,1],c = means, alpha=0.5, vmin=-60, vmax=60)
        plt.scatter(suggestions[:,0], suggestions[:,1], s=0.1, c='black', alpha=0.1)
        plt.xlim(domain[0,0], domain[0,1])
        plt.ylim(domain[1,0], domain[1,1])
        plt.colorbar(a)
        plt.savefig(gpcam_path+"mean_sugg" + str(iteration_counter).zfill(4))
        plt.show()
        ##write image to disc
        plt.figure(figsize=(20,5))
        a = plt.scatter(x_pred[:,0],x_pred[:,1],c = stds, alpha=0.5, vmin=-60, vmax=60)
        plt.scatter(suggestions[:,0], suggestions[:,1], s=0.1, c='black', alpha=0.1)
        plt.xlim(domain[0,0], domain[0,1])
        plt.ylim(domain[1,0], domain[1,1])
        plt.colorbar(a)
        plt.savefig(gpcam_path+"std_sugg" + str(iteration_counter).zfill(4))
        plt.show()
        print("Done!")
    
    

    print("Reading Chombo file. Iteration: ", iteration_counter)
    datasets, domain, global_x, global_y = read_file(chombo_path, filename, index, tol_ratio) ##immediately look for new data and read when available
    print("global dataset size: ", len(global_x))
    print("data:")
    print("filter tol: ", tol_ratio * np.max(global_y))
    plt.figure(figsize=(20,5))
    a = plt.scatter(global_x[:,0], global_x[:,1], c=global_y, alpha=.2)
    plt.colorbar(a)
    plt.show()
    print("done!")

    
    #UPDATE GPs
    print("Updating GP agents...")
    update_futures = [] 
    for ID in datasets:
        #comm data to kernel func.
        if len(datasets[ID][0])>4:
            kernels[ID] = partial(kernelPDE, x_data = global_x, y_data = global_y)
            set_new_kernel(client, kernels[ID], GPs[ID])
            set_args(client, GPs[ID], {"active": True})
            update_futures.append(tell(client, datasets[ID][0], datasets[ID][1], np.ones(datasets[ID][1].shape) * 0.1, GPs[ID]))
            
        else:
            kernels[ID] = partial(kernelPDE, x_data = global_x, y_data = global_y)
            set_new_kernel(client, kernels[ID], GPs[ID])
            set_args(client, GPs[ID], {"active": False})
            update_futures.append(tell(client, init_x_data, init_y_data, np.ones(init_y_data.shape) * 0.1, GPs[ID]))
    client.gather(update_futures)
    print("Updating GP agents done!")

    #TRAINING
    if iteration_counter in training_at:
        print("Training...")
        train_futures = []
        for ID in datasets: 
            if not GPs[ID].result().args["active"]: continue
            train_futures.append(train(client, hyperparameter_bounds, GPs[ID],  max_iter = 100, method = "mcmc"))
        client.gather(train_futures)
        print("Training done!")
    print("++++++++++++++++++++++++++++++++++")
    

## Building intuition of our kernel

In [ ]:
from itertools import product
x_pred1D = np.linspace(0,1,1000).reshape(-1,1)

x = np.linspace(0,600,1000)
def f1(x):
    return np.sin(5. * x) + np.cos(10. * x) + (2.* (x-0.4)**2) * np.cos(100. * x)

x_data = np.linspace(0,1,20).reshape(20,1)
y_data = f1(x_data[:,0]) + (np.random.rand(len(x_data))-0.5) * 0.1

plt.figure(figsize = (15,5))
plt.xticks([0.,0.5,1.0])
plt.yticks([-2,-1,0.,1])
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.plot(x_pred1D,f1(x_pred1D), color = 'orange', linewidth = 4)
plt.scatter(x_data[:,0],y_data, color = 'black')

In [ ]:
from scipy.interpolate import griddata
from gpcam.kernels import *
from scipy.interpolate import RBFInterpolator
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import spsolve
from scipy.spatial.distance import cdist

# Wendland C^2 compactly supported kernel (support radius = 1)
def wendland_c2(r):
    out = np.zeros_like(r)
    mask = r < 1.0
    rm = 1 - r[mask]
    out[mask] = rm**4 * (4*r[mask] + 1)
    return out


# Derivative of Wendland C^2 w.r.t. r
def wendland_c2_prime(r):
    out = np.zeros_like(r)
    mask = r < 1.0
    rm = 1 - r[mask]
    # φ'(r) = d/dr [ (1-r)^4 * (4r +1) ]
    #       = -4*(1-r)^3*(4r+1) + (1-r)^4 * 4
    out[mask] = -4 * rm**3 * (4*r[mask] + 1) + 4 * rm**4
    return out

# Simple RBF interpolator with Wendland kernel
class WendlandRBF:
    def __init__(self, x, y, epsilon=0.5):
        self.x = np.asarray(x)
        self.y = np.asarray(y)
        self.epsilon = epsilon
        r = cdist(self.x, self.x) * epsilon
        K = wendland_c2(r)
        K = csr_matrix(K)
        nnz = K.nnz

        # Total number of entries
        total = np.prod(K.shape)

        # Sparsity = fraction of zeros
        sparsity = nnz / total
        self.w = spsolve(K, self.y)

    def __call__(self, xnew):
        r = cdist(np.atleast_2d(xnew), self.x) * self.epsilon
        K = wendland_c2(r)
        K = csr_matrix(K)
        return np.asarray(K @ self.w)
        
    def gradient(self, xnew):
        """
        Compute the gradient of the RBF interpolant at one or multiple query points.
        xnew: array of shape (M, d)
        Returns: array of shape (M, d)
        """
        xnew = np.atleast_2d(xnew)  # (M, d)
        M, d = xnew.shape
        N = self.x.shape[0]
    
        grads = np.zeros((M, d))
    
        for j in range(M):
            diffs = xnew[j] - self.x        # (N, d)
            r = np.linalg.norm(diffs, axis=1) * self.epsilon  # (N,)
            phi_prime = wendland_c2_prime(r)
    
            grad = np.zeros(d)
            for i in range(N):
                if 0 < r[i] < 1.0:
                    grad += self.w[i] * phi_prime[i] * self.epsilon * (diffs[i] / (r[i] + 1e-12))
    
            grads[j] = grad
    
        return np.linalg.norm(grads, axis = 1)


def kernelPDE(x1, x2, hps, x_data = None, y_data = None):
    #st = time.time()
    d = get_distance_matrix(x1,x2)
    rbf = WendlandRBF(x_data, y_data, epsilon = hps[2])
    #print(time.time() - st, flush = True)
    #func1 = abs(rbf(x1))
    #func2 = abs(rbf(x2))

    func1 = rbf.gradient(x1)
    func2 = rbf.gradient(x2)
    
    k = hps[0] * np.outer(func1,func2) * matern_kernel_diff1(d,hps[1])
    return k 

def meanf(x,hps):
    return np.zeros(len(x))



kernel = partial(kernelPDE, x_data = x_data, y_data = y_data)
init_hyperparameters = np.array([1., 20., 0.05])
hyperparameter_bounds = np.array([[0.1, 10.],
                                  [0.1, 10.],
                                  [0.1, 0.5],])

my_gp1 = GPOptimizer(x_data, y_data, noise_variances=np.ones(y_data.shape) * 0.01,
                          kernel_function=kernel,
                          prior_mean_function=meanf,
                          init_hyperparameters = init_hyperparameters,
                          args={"active": True}, calc_inv=True)
my_gp1.train(hyperparameter_bounds=hyperparameter_bounds)

In [ ]:
#let's make a prediction
x_pred = x_pred1D.flatten()

mean1 = my_gp1.posterior_mean(x_pred.reshape(-1,1))["m(x)"]
var1 =  my_gp1.posterior_covariance(x_pred.reshape(-1,1), variance_only=False, add_noise=True)["v(x)"]
plt.figure(figsize = (16,10))
plt.plot(x_pred,mean1, label = "posterior mean", linewidth = 4)
plt.plot(x_pred1D,f1(x_pred1D), label = "latent function", linewidth = 4)
plt.fill_between(x_pred, mean1 - 3. * np.sqrt(var1), mean1 + 3. * np.sqrt(var1), alpha = 0.5, color = "grey", label = "var")
plt.scatter(x_data,y_data, color = 'black')


##looking at some validation metrics
print(my_gp1.rmse(x_pred1D,f1(x_pred1D).reshape(1000)))
print(my_gp1.crps(x_pred1D,f1(x_pred1D).reshape(1000)))

In [ ]:
rbf = WendlandRBF(x_data, y_data, epsilon = 0.4487)
func1 = abs(rbf.gradient(x_pred1D))

In [ ]:
plt.plot(x_pred1D,func1, label = "interpolant", linewidth = 4)